In [1]:
#-----------------------------------#
# Target: for preparing the test data before runing trained FOST model
# FOST only receives .csv
# DATA: IMFS and smapi within Huai river Plain 
# This practice is to remove all the invalid grids (for reducing computation) 
# author: X.W.
# 6-26-2022
#-----------------------------------#
import xarray as xr
import pandas as pd
#import geopandas as gpd
import numpy as np
import os
import glob

In [2]:
# 读取文件名的函数
def read_file(path, file_type):  # path 是指需要提取的文件夹所在路径;file_type是指文件的拓展名;txt格式的文件则输入‘.txt’
    filenames = os.listdir(path)
    filenames1 = []
    for filename in filenames:
        if os.path.splitext(filename)[1] == file_type:
            filenames1.append(filename)
    return filenames1

In [4]:
import numpy as np

def pad(value):
    try:
        return '{0:0>2}'.format(int(value))
    except:
        return value

vfunc = np.vectorize(pad)
# arr = vfunc(month)
# print(arr)

In [14]:
os.chdir(r'E:\Data_inputs\masked_data_HRB')  # 更改工作路径
curr_path = os.getcwd()  # 获取当前路径
files = read_file(curr_path, '.nc')  # 利用上述函数获取当前文件夹内的.nc文件名
# 最后一个文件是smapi，注意区分
files

['imfs0_HRB_from_1980_to_2021.nc',
 'imfs1_HRB_from_1980_to_2021.nc',
 'imfs2_HRB_from_1980_to_2021.nc',
 'imfs3_HRB_from_1980_to_2021.nc',
 'imfs4_HRB_from_1980_to_2021.nc',
 'imfs5_HRB_from_1980_to_2021.nc',
 'imfs6_HRB_from_1980_to_2021.nc',
 'smapi_HRB_from_1980_to_2021.nc']

In [ ]:
t_end = pd_date[(pd_date.date.dt.month == jj) & (pd_date.date.dt.year == ii)].index.values[0]

In [34]:
# select the data from specific month
pd_date = pd.DataFrame({'date':pd.date_range(start='1980-1-6', periods=2140, freq='7D')})
#pd.DataFrame({'date':pd.date_range(start='1979-12-31', periods=2140, freq='7D')})
month = np.arange(1,13)
year = np.arange(2019,2021)
ds = xr.open_dataset(files[0])
for kk in range(len(files)):
    os.makedirs('E:\\Data_inputs\\fost_test_in_VAL\\'+'val_input_imfs_'+str(kk))
    data = xr.open_dataset(files[kk])
    if kk == 7:
        dt = data['smapi'].values
    else: 
        dt = data['imf'].values
    df = pd.DataFrame(dt.reshape((dt.shape[0],-1)))
    id_nan = pd_date[pd_date.columns[pd_date.isnull().any()]].columns
    # 获得均为nan的列，df_new = df.iloc[:,id_nan.values]
    df = df.dropna(axis=1, how='all')
    for ii in year:
        for jj in month:
            t_end = pd_date[(pd_date.date.dt.month == jj) & (pd_date.date.dt.year == ii)].index.values[0]
            #t2 = pd_date[(pd_date.date.dt.month == jj) & (pd_date.date.dt.year == ii)].index.values[-1]
            df_val = df.loc[t_end-21:t_end-1,:]
            df_date = pd.DataFrame()
            df_date['Date'] = data.time[t_end-21:t_end]
            n = df_val.shape[1]
            df_date = pd.concat([df_date] * n, axis=1, ignore_index=True)
            df_val = pd.melt(df_val)
            df_date = pd.melt(df_date)
            df_val['Date'] = df_date.value
#             df_val = df.loc[t_end-21:t_end-1,:]
#             df_date = pd.DataFrame()
#             df_date['Date'] = data.time[t_end-20:t_end+1]
#             n = df_val.shape[1]
#             df_date = pd.concat([df_date] * n, axis=1, ignore_index=True)
#             df_val = pd.melt(df_val)
#             df_date = pd.melt(df_date)
#             df_val['Date'] = data.time[t_end-21:t_end]
            df_val.rename(columns = {'variable':'Node','value':'TARGET'}, inplace = True)
            # save to directed folder
            df_val.to_csv('E:\\Data_inputs\\fost_test_in_VAL\\'+'val_input_imfs_'
                          +str(kk)+'\\'
                          'imf'+str(kk)+'_HRB_pd_melt_'+str(ii)+'-'+str(vfunc(jj))+'.csv')
            

#df[(df.date.dt.month == 3) & (df.date.dt.year == 2019)].index.values[0]